In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/Shared drives/Data Mining Group 2/Utils/jay_utils.py" ./utils.py

In [ ]:
!pip install python-dotenv
!pip install catboost

     |████████████████████████████████| 66.3MB 57kB/s 


In [ ]:
%matplotlib inline

import os
import time

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils

plt.rcParams['figure.figsize'] = (20, 20)


from tqdm import tqdm_notebook
from tqdm import notebook
from tqdm.notebook import tqdm_notebook
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix

In [ ]:
DRIVE_DIR = "/content/drive/Shared drives/Data Mining Group 2/Dataset/unzipped_for_colab"
AUDIO_DIR = DRIVE_DIR + "/fma_medium/"

tracks_ld = utils.load(DRIVE_DIR + '/fma_metadata/tracks.csv')
features_ld = utils.load(DRIVE_DIR + '/fma_metadata/features.csv')
# echonest_ld = utils.load(DRIVE_DIR + '/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features_ld.index, tracks_ld.index)
# assert echonest_ld.index.isin(tracks_ld.index).all()

# tracks_ld.shape, features_ld.shape, echonest_ld.shape
tracks_ld.shape, features_ld.shape

((106574, 52), (106574, 518))

SUBSET

In [ ]:
subset = tracks_ld.index[tracks_ld['set', 'subset'].isin(['small', 'medium'])]

assert subset.isin(tracks_ld.index).all()
assert subset.isin(features_ld.index).all()


# features_all = features.join(echonest, how='inner').sort_index(axis=1)
# print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks_ld.loc[subset]
features_all = features_ld.loc[subset]

tracks.shape, features_all.shape

((25000, 52), (25000, 518))

In [ ]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']
# train_sm = tracks_sm.index[tracks_sm['set', 'split'] == 'training']
# val_sm = tracks_sm.index[tracks_sm['set', 'split'] == 'validation']
# test_sm = tracks_sm.index[tracks_sm['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))
# print('Small + Medium: {} training examples, {} validation examples, {} testing examples'.format(*map(len, [train_sm, val_sm, test_sm])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
# genres_sm = list(LabelEncoder().fit(tracks_sm['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
# print('Small + Medium: Top genres ({}): {}'.format(len(genres_sm), genres_sm))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
# genres_sm = list(MultiLabelBinarizer().fit(tracks_sm['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))
# print('All genres ({}): {}'.format(len(genres_sm), genres_sm))

19922 training examples, 2505 validation examples, 2573 testing examples
Top genres (16): ['Blues', 'Classical', 'Country', 'Easy Listening', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']
All genres (151): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 37, 38, 41, 42, 43, 45, 46, 47, 49, 53, 58, 63, 64, 65, 66, 70, 71, 74, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 97, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 137, 138, 166, 167, 169, 171, 172, 174, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 311, 314, 322, 337, 359, 360, 361, 362, 374, 378, 400, 401, 404, 428, 439, 440, 441, 442, 443, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 651, 659, 695, 741, 763, 808, 810, 811, 906, 1032, 1060, 1193, 1235]


Pre-processing

In [ ]:
def pre_process(tracks, features, columns, multi_label=False, verbose=False):
    if not multi_label:
        # Assign an integer value to each genre.
        enc = LabelEncoder()
        labels = tracks['track', 'genre_top']
        #y = enc.fit_transform(tracks['track', 'genre_top'])
    else:
        # Create an indicator matrix.
        enc = MultiLabelBinarizer()
        labels = tracks['track', 'genres_all']
        #labels = tracks['track', 'genres']

    # Split in training, validation and testing sets.
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
    X_train = features.loc[train, columns].to_numpy()
    X_val = features.loc[val, columns].to_numpy()
    X_test = features.loc[test, columns].to_numpy()
    
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    
    # Standardize features by removing the mean and scaling to unit variance.
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test, enc

Classification into multiple genres

In [ ]:
def test_classifiers_features(classifiers, feature_sets, multi_label=False):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores_m = pd.DataFrame(columns=columns, index=feature_sets.keys())
    # scores_sm = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        print(fset)
        y_train_m, y_val_m, y_test_m, X_train_m, X_val_m, X_test_m, enc_m = pre_process(tracks, features_all, fset, multi_label)
        # y_train_sm, y_val_sm, y_test_sm, X_train_sm, X_val_sm, X_test_sm, enc_sm = pre_process(tracks_sm, features_all_sm, fset, multi_label)
        scores_m.loc[fset_name, 'dim'] = X_train_m.shape[1]
        # scores_sm.loc[fset_name, 'dim'] = X_train_sm.shape[1]
        for clf_name, clf in classifiers.items():  # tqdm_notebook(classifiers.items(), desc='classifiers', leave=False):
            t = time.process_time()
            clf.fit(X_train_m, y_train_m)
            score_m = clf.score(X_test_m, y_test_m)
            y_pred_m = clf.predict(X_test_m)
            print(y_pred_m.shape, y_test_m.shape)
            # cm_m = confusion_matrix(y_test_m, y_pred_m)
            # cm_display_m = ConfusionMatrixDisplay(cm_m, display_labels=enc_m.classes_).plot()
            # cm_display_m.ax_.set_title(f'Medium: {clf_name} ({fset_name})')
            scores_m.loc[fset_name, clf_name] = score_m
            # print('Medium')
            # plot_confusion_matrix(clf, X_test_m, y_test_m)
            # plt.show()

            # clf.fit(X_train_sm, y_train_sm)
            # score_sm = clf.score(X_test_sm, y_test_sm)
            # y_pred_sm = clf.predict(X_test_sm)
            # cm_sm = confusion_matrix(y_test_sm, y_pred_sm)
            # cm_display_sm = ConfusionMatrixDisplay(cm_sm, display_labels=enc_sm.classes_).plot()
            # cm_display_sm.ax_.set_title(f'Small + Medium: {clf_name} ({fset_name})')
            # # print('Small + Medium')
            # # plot_confusion_matrix(clf, X_test_sm, y_test_sm)
            # plt.show()
            # scores_sm.loc[fset_name, clf_name] = score_sm
            times.loc[fset_name, clf_name] = time.process_time() - t
    # return scores_m, scores_sm, times
    return scores_m, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [ ]:
classifiers = {
    #LogisticRegression(),
    # 'LR': OneVsRestClassifier(LogisticRegression()),
    # 'SVC': OneVsRestClassifier(SVC()),
    # 'MLP': MLPClassifier(max_iter=700),
    'CatBoost_30' : CatBoostClassifier(n_estimators=30, logging_level='Silent'),
    'CatBoost_100' : CatBoostClassifier(n_estimators=100, logging_level='Silent'),
    'CatBoost_300' : CatBoostClassifier(n_estimators=300, logging_level='Silent'),
    'CatBoost_1000' : CatBoostClassifier(n_estimators=1000, logging_level='Silent'),
    'CatBoost_3000' : CatBoostClassifier(n_estimators=3000, logging_level='Silent')
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
    # 'mfcc': 'mfcc',
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    # 'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    # 'rolloff/rmse/zcr': ['spectral_rolloff', 'rmse', 'zcr']
}

scores_m, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores_m))
# ipd.display(format_scores(scores_sm))
ipd.display(times.style.format('{:.4f}'))

['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz']
(2573, 1) (2573,)
(2573, 1) (2573,)
(2573, 1) (2573,)
(2573, 1) (2573,)
(2573, 1) (2573,)



,dim,CatBoost_30,CatBoost_100,CatBoost_300,CatBoost_1000,CatBoost_3000
mfcc/contrast/chroma/centroid/tonnetz,322.000000,56.82%,59.81%,62.30%,62.92%,63.27%


,CatBoost_30,CatBoost_100,CatBoost_300,CatBoost_1000,CatBoost_3000
mfcc/contrast/chroma/centroid/tonnetz,116.6383,375.7163,1127.4089,3824.3009,11203.1132


In [ ]:
from IPython.display import HTML
display(HTML('''
<style>
  pre {
      white-space: normal;
  }
</style>
'''))
print("x\n" * 200)

x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x



In [ ]:
### If using the GPU version

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
### If using the TPU version

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#@title Keras baseline model CNN
import tensorflow as tf
from keras import backend as K

utils.quiet=True

model = keras.models.Sequential()
model.add(Reshape((*loader.shape, 1),  input_shape=loader.shape))
print(model.output_shape)

model.add(Conv2D(3, (13, 10), strides=(1, 4)))  # Subsample renamed to strides, rows/col => single kernel_size
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(15, (1, 10), strides=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(65, (1, 10), strides=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Flatten())
print(model.output_shape)
model.add(Dense(labels_onehot.shape[1]))
model.add(Activation("softmax"))
print(model.output_shape)

model.summary()

optimizer = keras.optimizers.SGD(1e-3)#lr=0.01, momentum=0.9, nesterov=True)
#optimizer = keras.optimizers.Adam()#lr=1e-5)#
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(SampleLoader(train, batch_size=16), steps_per_epoch=train.size, epochs=20, **params)
loss = model.evaluate_generator(SampleLoader(val, batch_size=16), val.size, **params)
loss = model.evaluate_generator(SampleLoader(test, batch_size=16), test.size, **params)
#Y = model.predict_generator(loader, test.size, pickle_safe=True, nb_worker=NB_WORKER, max_q_size=5)

loss